In [1]:
# from graphframes import *
from pyspark import SparkContext
from pyspark import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)

In [2]:
edge = sqlContext.read.csv("edge.csv", header = True)
node = sqlContext.read.csv("node.csv", header = True)

In [3]:
edge.show()
node.show()

+---+----+------------------+
|src| dst|        similarity|
+---+----+------------------+
|  1| 280|0.9569020683327998|
|  1| 997|0.9565269894180614|
|  1|1236|0.9567461501752356|
|  6|  57|0.9623191850941665|
|  6|  82|0.9661722872652426|
|  6| 131|0.9605763368855117|
|  6| 259|0.9578480816300617|
|  6| 280|0.9664919540417708|
|  6| 339|0.9560469401205195|
|  6| 345| 0.956438250975529|
|  6| 350|0.9686053837071976|
|  6| 463|0.9585969635136617|
|  6| 506|0.9506763783946054|
|  6| 537|0.9694831585397952|
|  6| 541|0.9670311309886417|
|  6| 584| 0.966651434080568|
|  6| 587|0.9808524669446692|
|  6| 646|0.9656450115230039|
|  6| 733|0.9649272899386095|
|  6| 735|0.9564413495860201|
+---+----+------------------+
only showing top 20 rows

+---+--------------------+--------------------+
| id|               title|                 url|
+---+--------------------+--------------------+
|  0|Test article for ...|https://en.wikine...|
|  1|Another test arti...|https://en.wikine...|
|  2|Copenhage

**show the graph**

In [ ]:
from graphframes import *
g = GraphFrame(node, edge)
g.vertices.show()
g.edges.show()

**PageRank**

In [ ]:
resultsPR = g.pageRank(resetProbability=0.15, tol=0.01)
resultsPR.vertices.select("id", "pagerank").show() 
resultsPR.edges.select("src", "dst", "weight").show()

**Connected Components**

In [ ]:
sc.setCheckpointDir("~/hw/hw3")
resultsCN = g.connectedComponents()
resultsCN.select("id","component").show()

**Triangle Counting**

In [ ]:
resultsTC = g.triangleCount()
resultsTC.select("id", "count").show()

In [ ]:
# resultsPR.join(resultsCN, resultsPR.id == resultsCN.id) graphframe has no attribute join
id = np.array(resultsPR.vertices.select("id","pagerank").collect())
id = resultsPR.vertices.select("id","pagerank").toPandas()
com = resultsCN.select("id","component").toPandas()
id.id = id.id.apply(int)
com.id = com.id.apply(int)
node_df = id.join(com, on = 'id', lsuffix = 'id', rsuffix = 'rid')
node_df = node_df.drop(['idrid'],axis = 1)
# rename not work
node_df = node_df.rename({0:'ID',1:'PageRank',2:'Component'},axis = "columns") 
node_df.to_csv("node_f.csv",sep = ",", index = False, encoding = "utf-8")